In [ ]:
import pandas as pd
import pymssql

# Azure SQL Server Configuration
SERVER = 'almagraby.database.windows.net'
DATABASE = 'Olist_dataset'
USERNAME = 'DEPI_Projecte'
PASSWORD = 'Almaghraby@240'
TABLE_NAME = 'olist_geolocation_dataset_olist_geolocation_dataset'

def get_connection():
    """Create database connection"""
    return pymssql.connect(
        server=SERVER,
        user=USERNAME,
        password=PASSWORD,
        database=DATABASE,
        port=1433,
        tds_version='7.4',
        timeout=60,
        charset='UTF-8'  # Important for encoding
    )

def fix_encoding(text):
    """
    Fix common encoding issues where UTF-8 was incorrectly decoded as Latin-1
    
    Examples:
    - 'abarÃ©' → 'abaré'
    - 'bÃºzios' → 'búzios'
    - 'algodÃµes' → 'algodões'
    """
    if pd.isna(text) or text == '':
        return text
    
    try:
        # Try to encode as Latin-1 and decode as UTF-8
        # This fixes the common "UTF-8 decoded as Latin-1" issue
        fixed = text.encode('latin-1').decode('utf-8')
        return fixed
    except (UnicodeDecodeError, UnicodeEncodeError):
        # If that fails, return original
        return text

def detect_garbled_characters(text):
    """
    Detect if text contains garbled characters
    Returns True if garbled, False otherwise
    """
    if pd.isna(text) or text == '':
        return False
    
    # Common garbled patterns
    garbled_patterns = ['Ã', 'Â', 'Ã©', 'Ã¡', 'Ã³', 'Ã£', 'Ãº', 'Ã§']
    
    for pattern in garbled_patterns:
        if pattern in text:
            return True
    
    return False

def main():
    print("=" * 70)
    print("Character Encoding Fix Tool")
    print("=" * 70)
    
    # Connect to database
    print("\n🔌 Connecting to database...")
    conn = get_connection()
    cursor = conn.cursor()
    print("✓ Connected!")
    
    # Read data with potential encoding issues
    print(f"\n📖 Reading data from {TABLE_NAME}...")
    query = f"""
    SELECT DISTINCT geolocation_city
    FROM {TABLE_NAME}
    WHERE geolocation_city IS NOT NULL
    AND geolocation_city != ''
    """
    df = pd.read_sql(query, conn)
    
    print(f"✓ Read {len(df):,} unique cities")
    
    # Detect garbled characters
    print("\n🔍 Detecting garbled characters...")
    df['is_garbled'] = df['geolocation_city'].apply(detect_garbled_characters)
    garbled_cities = df[df['is_garbled']]
    
    print(f"✓ Found {len(garbled_cities):,} cities with encoding issues")
    
    if len(garbled_cities) == 0:
        print("\n✅ No encoding issues found!")
        cursor.close()
        conn.close()
        return
    
    # Show examples
    print("\n📋 Examples of garbled text:")
    print("-" * 50)
    for idx, row in garbled_cities.head(20).iterrows():
        original = row['geolocation_city']
        fixed = fix_encoding(original)
        print(f"  {original:30} → {fixed}")
    
    # Ask user if they want to fix
    print("\n" + "=" * 70)
    print(f"Found {len(garbled_cities):,} cities with encoding issues")
    print("=" * 70)
    
    # Create fixed column
    print("\n🔧 Creating fixed version...")
    df['geolocation_city_fixed'] = df['geolocation_city'].apply(fix_encoding)
    
    # Check if fix column exists
    print("\n📋 Checking if 'geolocation_city_fixed' column exists...")
    check_column_query = f"""
    IF NOT EXISTS (
        SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
        WHERE TABLE_NAME = '{TABLE_NAME}' 
        AND COLUMN_NAME = 'geolocation_city_fixed'
    )
    BEGIN
        ALTER TABLE {TABLE_NAME}
        ADD geolocation_city_fixed NVARCHAR(255)
    END
    """
    cursor.execute(check_column_query)
    conn.commit()
    print("✓ Column ready")
    
    # Update database
    print(f"\n💾 Updating database with fixed encodings...")
    updated = 0
    
    for idx, row in garbled_cities.iterrows():
        original = row['geolocation_city']
        fixed = fix_encoding(original)
        
        if original != fixed:
            try:
                update_query = f"""
                UPDATE {TABLE_NAME}
                SET geolocation_city_fixed = %s
                WHERE geolocation_city = %s
                """
                cursor.execute(update_query, (fixed, original))
                updated += 1
                
                if updated % 100 == 0:
                    conn.commit()
                    print(f"  Updated {updated}/{len(garbled_cities)} cities...")
            except Exception as e:
                print(f"  Error updating '{original}': {e}")
    
    conn.commit()
    print(f"✓ Updated {updated:,} cities")
    
    # Show results
    print("\n📊 Sample of fixed data:")
    print("-" * 70)
    sample_query = f"""
    SELECT TOP 20 
        geolocation_city as original,
        geolocation_city_fixed as fixed
    FROM {TABLE_NAME}
    WHERE geolocation_city_fixed IS NOT NULL
    AND geolocation_city != geolocation_city_fixed
    """
    sample_df = pd.read_sql(sample_query, conn)
    print(sample_df.to_string(index=False))
    
    cursor.close()
    conn.close()
    
    print("\n" + "=" * 70)
    print("✅ ENCODING FIX COMPLETED!")
    print("=" * 70)
    print(f"Total cities processed: {len(df):,}")
    print(f"Cities with encoding issues: {len(garbled_cities):,}")
    print(f"Cities fixed: {updated:,}")
    print(f"\nFixed data stored in column: geolocation_city_fixed")
    print("=" * 70)

if __name__ == "__main__":
    main()

Character Encoding Fix Tool

🔌 Connecting to database...
✓ Connected!

📖 Reading data from olist_geolocation_dataset_olist_geolocation_dataset...


C:\Users\youse\AppData\Local\Temp\ipykernel_3696\1922755343.py:81: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✓ Read 8,010 unique cities

🔍 Detecting garbled characters...
✓ Found 2,085 cities with encoding issues

📋 Examples of garbled text:
--------------------------------------------------
  4Âº centenario                 → 4º centenario
  Ã¡gua boa                      → água boa
  Ã¡gua branca                   → água branca
  Ã¡gua clara                    → água clara
  Ã¡gua comprida                 → água comprida
  Ã¡gua doce                     → água doce
  Ã¡gua doce do maranhÃ£o        → água doce do maranhão
  Ã¡gua doce do norte            → água doce do norte
  Ã¡gua fria                     → água fria
  Ã¡gua fria de goiÃ¡s           → água fria de goiás
  Ã¡gua preta                    → água preta
  Ã¡guas belas                   → águas belas
  Ã¡guas da prata                → águas da prata
  Ã¡guas de chapecÃ³             → águas de chapecó
  Ã¡guas de lindÃ³ia             → águas de lindóia
  Ã¡guas de sÃ£o pedro           → águas de são pedro
  Ã¡guas de santa bÃ¡rbar